In [ ]:
import ta
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.trend import MACD, ADXIndicator
from ta.volatility import BollingerBands, AverageTrueRange
from ta.volume import ChaikinMoneyFlowIndicator, VolumeWeightedAveragePrice
from ta.trend import EMAIndicator

In [1]:

def relevant_indicators(df):
    indicators = pd.DataFrame(index=df.index)
    indicators['Open'] = df['Open']
    indicators['High'] = df['High']
    indicators['Low'] = df['Low']
    indicators['Adj Close'] = df['Adj Close']
    indicators['Volume'] = df['Volume']
    
    # MACD
    # macd_12_26_9 = MACD(close=df['Close'], window_slow=26, window_fast=12, window_sign=9)
    # indicators['MACD_12_26'] = macd_12_26_9.macd()
    # indicators['MACD_12_26_Signal'] = macd_12_26_9.macd_signal()
    
    macd_20_40_10 = MACD(close=df['Close'], window_slow=40, window_fast=20, window_sign=10)
    indicators['MACD_20_40'] = macd_20_40_10.macd()
    indicators['MACD_20_40_Signal'] = macd_20_40_10.macd_signal()
    
    macd_24_56_18 = MACD(close=df['Close'], window_slow=56, window_fast=24, window_sign=18)
    indicators['MACD_24_56'] = macd_24_56_18.macd()
    indicators['MACD_24_56_Signal'] = macd_24_56_18.macd_signal()
    
    # PPO (Percentage Price Oscillator)
    # ppo_12_26 = macd_12_26_9.macd_diff()
    # ppo_20_40 = macd_20_40_10.macd_diff()
    # ppo_24_56 = macd_24_56_18.macd_diff()
    # indicators['PPO_12_26'] = ppo_12_26
    # indicators['PPO_20_40'] = ppo_20_40
    # indicators['PPO_24_56'] = ppo_24_56
    
    # Bollinger Bands
    bollinger = BollingerBands(close=df['Close'], window=20, window_dev=2)
    indicators['Bollinger_High'] = bollinger.bollinger_hband()
    indicators['Bollinger_Low'] = bollinger.bollinger_lband()
    
    # ATR (Average True Range)
    # atr_14 = AverageTrueRange(high=df['High'], low=df['Low'], close=df['Close'], window=14).average_true_range()
    # atr_20 = AverageTrueRange(high=df['High'], low=df['Low'], close=df['Close'], window=20).average_true_range()
    # indicators['ATR_14'] = atr_14
    # indicators['ATR_20'] = atr_20
    
    # ADX (Average Directional Index)
    # adx_14 = ADXIndicator(high=df['High'], low=df['Low'], close=df['Close'], window=14).adx()
    adx_20 = ADXIndicator(high=df['High'], low=df['Low'], close=df['Close'], window=20).adx()
    # indicators['ADX_14'] = adx_14
    indicators['ADX_20'] = adx_20
    
    # Supertrend
    # the ta library does not provide a built-in supertrend indicator. Thus, definying an external function is needed.
    indicators['Supertrend_10_2'] = calculate_supertrend(df, atr_window=10, multiplier=2)
    indicators['Supertrend_10_3'] = calculate_supertrend(df, atr_window=10, multiplier=3)
    # indicators['Supertrend_14_2'] = calculate_supertrend(df, atr_window=14, multiplier=2)
    indicators['Supertrend_14_3'] = calculate_supertrend(df, atr_window=14, multiplier=3)
    
    return indicators

def calculate_supertrend(df, atr_window, multiplier):
    atr = AverageTrueRange(high=df['High'], low=df['Low'], close=df['Close'], window=atr_window).average_true_range()
    hl2 = (df['High'] + df['Low']) / 2
    final_upperband = hl2 + (multiplier * atr)
    final_lowerband = hl2 - (multiplier * atr)
    
    supertrend = pd.Series(index=df.index, dtype = 'float32')
    in_uptrend = True
    
    for i in range(1, len(df)):
        if df['Close'][i] > final_upperband[i-1]:
            in_uptrend = True
        elif df['Close'][i] < final_lowerband[i-1]:
            in_uptrend = False
        
        if in_uptrend:
            supertrend[i] = final_lowerband[i]
        else:
            supertrend[i] = final_upperband[i]
    
    return supertrend

In [1]:
# Function to normalize indicators using Robust Scaler
def normalize_indicators(indicators):
    scaler = RobustScaler()
    scaled_indicators = scaler.fit_transform(indicators)
    scaled_indicators_df = pd.DataFrame(scaled_indicators, index=indicators.index, columns=indicators.columns)
    return scaled_indicators_df